In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Hyperparameter
batch_size = 10
z_dim = 128
learning_rate_gen = 5e-5
learning_rate_dis = 5e-5
max_step = 20000
n_dis = 5
# lambda (gradient panelty)
lam = 120.

In [3]:
def residual_block(x, filters = 64):
    conv1 = tf.layers.conv2d(x, 
                             filters = filters,
                             kernel_size = 3,
                             strides = 1,
                             padding = 'same')
    conv1 = tf.layers.batch_normalization(conv1)
    conv1 = tf.nn.relu(conv1)
    
    conv2 = tf.layers.conv2d(conv1,
                             filters = filters,
                             kernel_size = 3,
                             strides = 1,
                             padding = 'same')
    conv2 = tf.layers.batch_normalization(conv2)
    
    return tf.add(x, conv2)

def pixel_shuffle(I, r):
    bsize, a, b, c = I.get_shape().as_list()
    X = tf.reshape(I, (bsize, a, b, r, r))
    X = tf.split(X, a, 1)  # a, [bsize, b, r, r]
    X = tf.concat([tf.squeeze(x) for x in X], 2)  # bsize, b, a*r, r
    X = tf.split(X, b, 1)  # b, [bsize, a*r, r]
    X = tf.concat([tf.squeeze(x) for x in X], 2)  # bsize, a*r, b*r
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def PS(X, r):
    # Main OP that you can arbitrarily use in you tensorflow code
    Xc = tf.split(X, X.get_shape()[3] // (r ** 2), 3)
    # if self.is_train:
    X = tf.concat([pixel_shuffle(x, r) for x in Xc], 3)

    #    X = tf.concat([pixel_shuffle_test(x, r) for x in Xc], 3)
    return X

def resize_block(x, filters):
    out = tf.layers.conv2d(x,
                           filters = filters,
                           kernel_size = 3,
                           strides = 1,
                           padding = 'same')
    out = PS(out, 2)
    out = tf.layers.batch_normalization(out)
    return tf.nn.relu(out)

In [4]:
# Define Generator Network
def generator(x, reuse = False):
    with tf.variable_scope('generator', reuse = reuse):
        fc1 = tf.layers.dense(x, 
                              units = 6 * 6 * 256, 
                              activation = tf.nn.leaky_relu)
        fc1 = tf.layers.batch_normalization(fc1)
        fc1 = tf.reshape(fc1, [-1, 6, 6, 256])

        # First Convolutional Layer
        conv1 = tf.layers.conv2d_transpose(fc1, 
                                           filters = 128, 
                                           kernel_size = 5,
                                           strides = 2,
                                           padding = 'same',
                                           activation = tf.nn.leaky_relu)
        conv1 = tf.layers.batch_normalization(conv1)

        res_blocks = []
        res_blocks.append(conv1)
        for i in range(6):
            res_blocks.append(residual_block(res_blocks[i], filters = 128))
            
            
        resize1 = resize_block(res_blocks[-1], filters = 128)
        resize2 = resize_block(resize1, filters = 64)
        resize3 = resize_block(resize2, filters = 32)
        
        # Third Convolutional Layer (Output Layer with tanh activation)
        output = tf.layers.conv2d_transpose(resize3, 
                                            filters = 3, 
                                            kernel_size = 9, 
                                            strides = 1,
                                            padding = 'same',
                                            activation = tf.nn.tanh)
        print (output)
        return output

# test
# z_input = tf.placeholder(tf.float32, shape = [None, z_dim], name = 'gen_input')
# generator(z_input)

In [5]:
def discriminator(x, reuse = False):
    with tf.variable_scope('discriminator', reuse = reuse):
        
        # conv1: 48 * 48 * 64
        conv1 = tf.layers.conv2d(x,
                                 filters = 64,
                                 kernel_size = 5,
                                 strides = 2,
                                 padding = 'same',
                                 activation = tf.nn.leaky_relu)
        conv1 = tf.layers.batch_normalization(conv1)
        
        
        # conv2: 24 * 24 * 64
        conv2 = tf.layers.conv2d(conv1,
                                 filters = 64,
                                 kernel_size = 5,
                                 strides = 2,
                                 padding = 'same',
                                 activation = tf.nn.leaky_relu)
        conv2 = tf.layers.batch_normalization(conv2)
        
        # Residual Block 1
        res1 = tf.layers.conv2d(conv2, 
                             filters = 64,
                             kernel_size = 3,
                             strides = 1,
                             padding = 'same',
                             activation = tf.nn.relu)
        res1 = tf.layers.batch_normalization(res1)
        res1_1 = tf.layers.conv2d(res1,
                                 filters = 64,
                                 kernel_size = 3,
                                 strides = 1,
                                 padding = 'same',
                                 activation = tf.nn.relu)
        res1_1 = tf.layers.batch_normalization(res1_1)
        
        res1_1 = tf.add(res1_1, conv2)
        
        # Residual Block 2
        res2 = tf.layers.conv2d(res1_1, 
                             filters = 64,
                             kernel_size = 3,
                             strides = 1,
                             padding = 'same',
                             activation = tf.nn.relu)
        res2 = tf.layers.batch_normalization(res2)
        res2_1 = tf.layers.conv2d(res2,
                                 filters = 64,
                                 kernel_size = 3,
                                 strides = 1,
                                 padding = 'same',
                                 activation = tf.nn.relu)
        res2_1 = tf.layers.batch_normalization(res2_1)
        res2_1 = tf.add(res2_1, res1_1)
        
        
        # res2_1 = residual_block(conv2, filters = 64)
        # res2_2 = residual_block(res2_1, filters = 64)
        
        # conv3: 12 * 12 * 64
        conv3 = tf.layers.conv2d(res2_1,
                                 filters = 64,
                                 kernel_size = 5,
                                 strides = 2,
                                 padding = 'same',
                                 activation = tf.nn.leaky_relu)
        
        conv3 = tf.layers.batch_normalization(conv3)
        
        # res3_1 = residual_block(conv3, filters = 64)
        # res3_2 = residual_block(res3_1, filters = 64)
        
        
        res3 = tf.layers.flatten(conv3)
        
        
        # fc1: 128 units
        fc1 = tf.layers.dense(res3, 
                              units = 128,
                              activation = tf.nn.leaky_relu)
        output = tf.layers.dense(fc1,
                                units = 1)
        return output

In [6]:
# Input
gen_input = tf.placeholder(tf.float32, shape = [batch_size, z_dim], name = 'gen_input')
dis_input = tf.placeholder(tf.float32, shape = [batch_size, 96, 96, 3], name = 'dis_input')
fake_input = generator(gen_input)

print (fake_input)

# Logits
true_logit = discriminator(dis_input)
fake_logit = discriminator(fake_input, reuse = True)
dis_loss = tf.reduce_mean(fake_logit - true_logit)
gen_loss = tf.reduce_mean(- fake_logit)

# Variables
gen_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
dis_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')

## Gradient Penalty

# Generate Interpolated Samples
uniform_dist = tf.contrib.distributions.Uniform(low=0., high=1.)
alpha = uniform_dist.sample((batch_size, 1, 1, 1))
interpolated = dis_input + alpha * (fake_input - dis_input)

# Calculate Gradients
int_logit = discriminator(interpolated, reuse = True)
gradients = tf.gradients(int_logit, [interpolated, ])[0]
gradients_l2 = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis = [1, 2, 3]))

# Panelty
gradient_penalty = lam * tf.reduce_mean(tf.square(gradients_l2 - 1.))
dis_loss += gradient_penalty

# Optimizer (RMSProp) RMSPropOptimizer
gen_opt = tf.train.AdamOptimizer(learning_rate_gen).minimize(gen_loss, var_list = gen_var)
dis_opt = tf.train.AdamOptimizer(learning_rate_dis).minimize(dis_loss, var_list = dis_var)

# Initialization
init = tf.global_variables_initializer()

resize1: Tensor("generator/Relu_6:0", shape=(10, 24, 24, 32), dtype=float32)
resize2: Tensor("generator/Relu_7:0", shape=(10, 48, 48, 16), dtype=float32)
resize3: Tensor("generator/Relu_8:0", shape=(10, 96, 96, 8), dtype=float32)
Tensor("generator/conv2d_transpose_1/Tanh:0", shape=(10, 96, 96, 3), dtype=float32)
Tensor("generator/conv2d_transpose_1/Tanh:0", shape=(10, 96, 96, 3), dtype=float32)
Instructions for updating:
Use the retry module or similar alternatives.


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

from data_utils_batch import *
data_X = dataReader('../data/animate/')

model_dir = '../model/'
saver = tf.train.Saver()

for i in range(max_step):
    
    for j in range(n_dis):
        batch_x = data_X.next_batch(batch_size)
        batch_x = batch_x * 0.9
        
        z = np.random.normal(0., 0.5, size=[batch_size, z_dim])
        feed_dict = {gen_input: z, dis_input: batch_x}
        sess.run([dis_opt], feed_dict = feed_dict)
        
    batch_x = data_X.next_batch(batch_size)
    batch_x = batch_x * 0.9
    
    z = np.random.normal(0., 0.5, size=[batch_size, z_dim])
    feed_dict = {gen_input: z, dis_input: batch_x}
    _, d_loss, g_loss, p = sess.run([gen_opt, dis_loss, gen_loss, gradient_penalty], feed_dict = feed_dict)
    if i % 200 == 0:
        print ('step {}: {}, {}, {}'.format(i, d_loss, g_loss, p))
        saver.save(sess, model_dir + 'wgan', global_step = i)

step 0: 76.09224700927734, -0.004992236848920584, 76.48567199707031
step 200: -77.4598159790039, -30.648754119873047, 7.34606409072876
step 400: -70.12602996826172, 2.745579957962036, 7.2963995933532715
step 600: -62.57015609741211, -0.05891614034771919, 7.75702428817749
step 800: -68.90266418457031, 50.14519500732422, 10.044238090515137
step 1000: -51.956974029541016, 39.60683059692383, 12.036355018615723
step 1200: -39.3770866394043, 62.28644943237305, 3.605725049972534
step 1400: -35.844425201416016, 96.47035217285156, 3.571990728378296
step 1600: -50.82112503051758, 59.412086486816406, 3.3764374256134033
step 1800: -31.55633544921875, 67.28782653808594, 1.2836687564849854
step 2000: -35.508811950683594, 8.882211685180664, 2.9375905990600586
step 2200: -31.189538955688477, 11.903844833374023, 2.4350178241729736
step 2400: -26.37350082397461, -14.782821655273438, 3.0499672889709473
step 2600: -37.594749450683594, 35.47789001464844, 3.8065853118896484
step 2800: -40.911827087402344, 1

In [1]:
# Generate images from noise, using the generator network.
cnt = 0

f, a = plt.subplots(8, 10, figsize=(10, 8))
for i in range(10):
    # Noise input.
    z = np.random.normal(0., 0.5, size=[batch_size, z_dim])
    g = sess.run(fake_input, feed_dict={gen_input: z})
    
    g[g > 0.9] = 0.9
    g[g < -0.9] = -0.9
    g = (g + 0.9) / 2. / 0.9
    
    batch_x = data_X.next_batch(batch_size)
    batch_x = (batch_x + 1.) / 2.
        
    for j in range(4):
        # Generate image from noise. Extend to 3 channels for matplot figure.
        cv2.imwrite('../result/{}.jpg'.format(str(cnt).zfill(3)), g[j] * 255)
        a[j][i].imshow(g[j])
        a[j + 4][i].imshow(batch_x[j])
        cnt += 1

f.show()
plt.draw()

NameError: name 'plt' is not defined